In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
original_tm = pd.read_csv('./data/transition_matrix.csv')
original_tm.head(5)

,state,next_state,prob
0,R_ANA,R_P,0.233380
1,R_ANA,S_ANA,0.000471
2,R_ANA,S_B!,0.017916
3,R_ANA,S_B#,0.036775
4,R_ANA,S_B+,0.075436


In [3]:
transition_matrix = original_tm.pivot(index='state', columns='next_state', values='prob').fillna(0)

In [4]:
transition_matrix

next_state,R_ANA,R_B!,R_B!.D=,R_B#,R_B+,R_B+.D#,R_B+.D#.S#,R_B+.D#.S#.ACF,R_B+.D#.S#.APP,R_B+.D#.S#.AX1,...,S_F-.S#.ANA,S_F-.S#.AV5,S_F-.S#.AV6,S_F-.S#.AX5,S_F-.S#.AX6,S_F-.S-,S_F/,S_F=,S_P,S_SV
state,,,,,,,,,,,,,,,,,,,,,
R_ANA,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000471,0.000471,0.073550,0.000471
R_B!,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
R_B!.D=,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000
R_B#,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
R_B+,0.000000,0.0,0.0,0.000000,0.000000,0.449228,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_F-.S-,0.120000,0.0,0.0,0.000000,0.040000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
S_F/,0.559701,0.0,0.0,0.014925,0.014925,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
S_F=,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [5]:
def calculate_absorption_probabilities(transition_matrix, absorbing_states):
    # transient and absorbing states
    all_states = list(transition_matrix.index)
    transient_states = [state for state in all_states if state not in absorbing_states]

    # Extract submatrices Q and R
    Q = transition_matrix.loc[transient_states, transient_states].values
    R = transition_matrix.loc[transient_states, absorbing_states].values

    # Calculate N = (I - Q)^(-1)
    I = np.eye(len(Q))
    N = np.linalg.inv(I - Q)

    # Calculate absorption probabilities B = N * R
    B = np.dot(N, R)

    # Convert results to a DataFrame
    absorption_probabilities = pd.DataFrame(B, index=transient_states, columns=absorbing_states)
    
    return absorption_probabilities, N

In [6]:
def calculate_expected_absorption_times(N, transient_states):
    # Calculate expected absorption times (state values) as row sums of N
    expected_times = N.sum(axis=1)

    # Convert results to a DataFrame for better readability
    expected_times_df = pd.DataFrame(expected_times, index=transient_states, columns=['Expected Absorption Time'])
    
    return expected_times_df

In [7]:
# "R_P", "S_P" are absorbing states
absorbing_states = ["R_P", "S_P"]
absorption_probabilities, N = calculate_absorption_probabilities(transition_matrix, absorbing_states)

# Calculate the expected absorption times using N
transient_states = [state for state in transition_matrix.index if state not in absorbing_states]
expected_absorption_times = calculate_expected_absorption_times(N, transient_states)

In [8]:
absorption_probabilities

,R_P,S_P
R_ANA,0.673193,0.326807
R_B!,0.000000,1.000000
R_B!.D=,0.000000,1.000000
R_B#,0.999892,0.000108
R_B+,0.573735,0.426265
...,...,...
S_F-.S#.AX6,0.326951,0.673049
S_F-.S-,0.594605,0.405395
S_F/,0.589154,0.410846
S_F=,1.000000,0.000000


In [14]:
absorption_probabilities.loc[['S_SV']]

,R_P,S_P
S_SV,0.581304,0.418696


In [9]:
absorption_probabilities.sum(axis=0)

R_P    178.711198
S_P    176.288802
dtype: float64

In [10]:
expected_absorption_times

,Expected Absorption Time
R_ANA,4.334093
R_B!,2.000000
R_B!.D=,1.000000
R_B#,1.341363
R_B+,7.858866
...,...
S_F-.S#.AX6,4.702150
S_F-.S-,6.993812
S_F/,5.701785
S_F=,1.000000


In [11]:
transition_matrix

next_state,R_ANA,R_B!,R_B!.D=,R_B#,R_B+,R_B+.D#,R_B+.D#.S#,R_B+.D#.S#.ACF,R_B+.D#.S#.APP,R_B+.D#.S#.AX1,...,S_F-.S#.ANA,S_F-.S#.AV5,S_F-.S#.AV6,S_F-.S#.AX5,S_F-.S#.AX6,S_F-.S-,S_F/,S_F=,S_P,S_SV
state,,,,,,,,,,,,,,,,,,,,,
R_ANA,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000471,0.000471,0.073550,0.000471
R_B!,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
R_B!.D=,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000
R_B#,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
R_B+,0.000000,0.0,0.0,0.000000,0.000000,0.449228,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_F-.S-,0.120000,0.0,0.0,0.000000,0.040000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
S_F/,0.559701,0.0,0.0,0.014925,0.014925,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
S_F=,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [12]:
transition_matrix.to_csv('./data/transition_matrix_expanded.csv', sep=';')

In [13]:
absorption_probabilities.to_csv('./data/absorption_probabilities.csv')